In [7]:
import numpy as np
from pymongo import MongoClient
from sklearn.cluster import KMeans


In [8]:
#MongoDB!
client=MongoClient('mongodb://localhost:27017/Photos')
db=client.Photos
collection=db.collection_En

In [9]:
path='C:/Users/ASUS/Desktop/tebourbi/cbir2/eh_descriptors'
table=[]
for i in range (1):
    with open(path+ '/'+'eh'+str(i+1)+'.txt','r') as f:
        lines = f.readlines()
        for j in lines:
            table.append(j.split())
arraytable=np.array(table)

path='C:/Users/ASUS/Desktop/tebourbi/cbir2/ht_descriptors'
table1=[]
for i in range (1):
    with open(path+ '/'+'ht'+str(i+1)+'.txt','r') as f:
        lines = f.readlines()
        for j in lines:
            table1.append(j.split())       
arraytable1=np.array(table1)
X=np.zeros((len(table),193))
for i in range(len(table)):
    for j in range(len(table[i])):
        X[i][j]=arraytable[i][j]

    for j in range(len(table1[i])):
        X[i][j+150]=arraytable1[i][j]


In [10]:
Kmeans=KMeans(n_clusters=20).fit(X)
Clusters=Kmeans.predict(X)
centroids = Kmeans.cluster_centers_
for k in range(0,1000):
    cluster=str(Clusters[k])
    line_DB={"index":k,"edge_value":table[k]+table1[k],"cluster":cluster}
    collection.insert_one(line_DB)

In [11]:
import os
from flask import Flask, request, render_template
from scipy.spatial import distance as dist
from PIL import Image

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''


#Distance 

def calcDistance(pic1,pic2):
    result=float(0)
    for i in range(0,len(pic1)):
        result+=(float(pic1[i])-float(pic2[i]))**2
    
    return result


In [ ]:
    
app = Flask(__name__)
@app.route('/', methods=['GET', 'POST'])
def index():
    client=MongoClient('mongodb://localhost:27017/Photos')
    db=client.Photos
    collection=db.collection_En
    if request.method == 'POST':
        file = request.files['query_img']
        img = Image.open(file.stream)  
        uploaded_img_path = "thumbnails" + file.filename
        img.save(uploaded_img_path)
        index = file.filename.split('.')[0]
        index = int (index)
        cluster=collection.find({"index":index})[0]["cluster"]
        cluster_picture_edge=[]
        for col in collection.find({"cluster":cluster}):
            cluster_picture_edge.append(col)
        similar_picture=[{"result":0,"index":cluster_picture_edge[0]["index"]}]
        for k in range(1,len(cluster_picture_edge)):
            result=calcDistance(cluster_picture_edge[0]["edge_value"],cluster_picture_edge[k]["edge_value"])
            similar_picture.append({"result":result,"index":cluster_picture_edge[k]["index"]})
        similar_picture=sorted(similar_picture,key=lambda x:x["result"])
        path='/thumbnails'
        answers =[] 
        for j in range(1,len(similar_picture)):
            if (j == 6): break
            answers.append([path+'/'+str(similar_picture[j]["index"])+'.jpg',' '])

        print(answers)
        return render_template('index.html',query_path=uploaded_img_path,answers=answers)
    else:
        return render_template('index.html')

if __name__=="__main__":

    app.run("127.0.0.1", debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2019-11-03 17:21:33,918] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-58-1143e59b94